Statistics analysis 
Raster files 
Max depth results

In [1]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
import os
from rasterio.mask import mask

# Understanding depth 

In [2]:
# Paths
raster_folder = r"E:\Project_deliverable\GIS\Flood_modeling_results"
polygon_path = r"E:\Project_deliverable\GIS\Watershed\subbasin_Cypress_Park\Subbasin_CP.shp"

In [6]:
# Load polygon
gdf = gpd.read_file(polygon_path)
geom = [gdf.geometry.unary_union]  # merge into single polygon

# Collect results
rows = []

for file in os.listdir(raster_folder):
    if file.endswith(".tif"):  # adjust if .img, .asc, etc.
        raster_path = os.path.join(raster_folder, file)
        
        with rasterio.open(raster_path) as src:
            out_image, _ = mask(src, geom, crop=True)
            data = out_image[0]  # first band
            
            # Remove NoData values
            nodata = src.nodata
            if nodata is not None:
                data = data[data != nodata]
            
            # Only calculate if data remains
            if data.size > 0:
                min_val = float(np.min(data))
                max_val = float(np.max(data))
                mean_val = float(np.mean(data))
                rows.append({"Raster": file, "Min": min_val, "Max": max_val, "Mean": mean_val})
            else:
                rows.append({"Raster": file, "Min": None, "Max": None, "Mean": None})

# Save results
df = pd.DataFrame(rows)
print(df)
#df.to_csv(os.path.join(raster_folder, "summary_stats.csv"), index=False)


                Raster       Min       Max      Mean
0  base_100yr_proc.tif  0.001008  7.298561  0.671344
1     base_2y_proc.tif  0.001013  4.787817  0.420476
2   base_10yr_proc.tif  0.001003  6.626595  0.518332


C:\Users\dveronez\AppData\Local\Temp\ipykernel_34452\3763619341.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = [gdf.geometry.unary_union]  # merge into single polygon


In [7]:
print(os.path.join(raster_folder, "summary_stats.csv"))

E:\Project_deliverable\GIS\Flood_modeling_results\summary_stats.csv


# Calculating storage area

## subbasin

In [24]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np

# ------------------------------------------------------------
# FILE PATHS
# ------------------------------------------------------------
subbasin_file = r"E:\Project_deliverable\GIS\Watershed\subbasin_Cypress_Park\Subbasin_CP.shp"  # subbasin polygon shapefile
depth_tif     = r"E:\Project_deliverable\GIS\Flood_modeling_results\base_10yr_proc.tif"

# ------------------------------------------------------------
# 1. Load subbasin polygon
# ------------------------------------------------------------
sub = gpd.read_file(subbasin_file)

# ------------------------------------------------------------
# 2. Load raster and reproject polygon to raster CRS
# ------------------------------------------------------------
with rasterio.open(depth_tif) as src:
    sub = sub.to_crs(src.crs)

# ------------------------------------------------------------
# 3. Clip depth raster using mask
# ------------------------------------------------------------
with rasterio.open(depth_tif) as src:
    out_image, out_transform = rasterio.mask.mask(src, sub.geometry, crop=True)
    depth_ft = out_image[0]

# Clean negative or no-data values
depth_ft = np.where(depth_ft < 0, 0, depth_ft)

# ------------------------------------------------------------
# 4. Compute cell area (in square feet)
# ------------------------------------------------------------
cell_size_x = out_transform[0]          # feet per pixel
cell_size_y = abs(out_transform[4])     # feet per pixel
cell_area_ft2 = cell_size_x * cell_size_y

# ------------------------------------------------------------
# 5. Compute total volume
# ------------------------------------------------------------
# cubic feet
volume_ft3 = np.sum(depth_ft * cell_area_ft2)

# acre-feet (1 AF = 43560 ft³)
volume_acft = volume_ft3 / 43560

# cubic meters (optional)
#volume_m3 = volume_ft3 * 0.0283168

print(f"Total Flood Volume: {volume_ft3} cubic feet")
print(f"Total Flood Volume: {volume_acft} acre-feet")
# print(f"Total Flood Volume: {volume_m3} cubic meters")

Total Flood Volume: 4385648.5 cubic feet
Total Flood Volume: 100.68063354492188 acre-feet


In [25]:
print("Transform:", out_transform)
print("Cell size X:", out_transform[0])
print("Cell size Y:", abs(out_transform[4]))


Transform: | 9.84, 0.00, 1723826.81|
| 0.00,-9.84, 153439.74|
| 0.00, 0.00, 1.00|
Cell size X: 9.842184614806197
Cell size Y: 9.842184614806197


## Cypress Park

In [22]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np

# ------------------------------------------------------------
# FILE PATHS
# ------------------------------------------------------------
subbasin_file = r"E:\Project_deliverable\GIS\Planning and Cadastre\Cypress_Park_boundaries\Cypress_Park_boundaries.shp"  # Cypress Park polygon shapefile
depth_tif     = r"E:\Project_deliverable\GIS\Flood_modeling_results\base_10yr_proc.tif"

# ------------------------------------------------------------
# 1. Load subbasin polygon
# ------------------------------------------------------------
sub = gpd.read_file(subbasin_file)

# ------------------------------------------------------------
# 2. Load raster and reproject polygon to raster CRS
# ------------------------------------------------------------
with rasterio.open(depth_tif) as src:
    sub = sub.to_crs(src.crs)

# ------------------------------------------------------------
# 3. Clip depth raster using mask
# ------------------------------------------------------------
with rasterio.open(depth_tif) as src:
    out_image, out_transform = rasterio.mask.mask(src, sub.geometry, crop=True)
    depth_ft = out_image[0]

# Clean negative or no-data values
depth_ft = np.where(depth_ft < 0, 0, depth_ft)

# ------------------------------------------------------------
# 4. Compute cell area (in square feet)
# ------------------------------------------------------------
cell_size_x = out_transform[0]          # feet per pixel
cell_size_y = abs(out_transform[4])     # feet per pixel
cell_area_ft2 = cell_size_x * cell_size_y

# ------------------------------------------------------------
# 5. Compute total volume
# ------------------------------------------------------------
# cubic feet
volume_ft3 = np.sum(depth_ft * cell_area_ft2)

# acre-feet (1 AF = 43560 ft³)
volume_acft = volume_ft3 / 43560

# cubic meters (optional)
#volume_m3 = volume_ft3 * 0.0283168

print(f"Total Flood Volume: {volume_ft3} cubic feet")
print(f"Total Flood Volume: {volume_acft} acre-feet")
# print(f"Total Flood Volume: {volume_m3} cubic meters")

Total Flood Volume: 159231.0625 cubic feet
Total Flood Volume: 3.655442237854004 acre-feet


In [27]:
%pip install fiona shapely pyproj

  Using cached fiona-1.10.1-cp312-cp312-win_amd64.whl.metadata (58 kB)
Using cached fiona-1.10.1-cp312-cp312-win_amd64.whl (24.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [28]:
import fiona
import shapely.geometry as geom
import shapely.ops as ops
from pyproj import Transformer

def calc_area(shp_path):
    # Open shapefile
    with fiona.open(shp_path) as src:
        crs = src.crs

        # Reproject to Maryland StatePlane Feet (EPSG:2248)
        transformer = Transformer.from_crs(crs, "EPSG:2248", always_xy=True)

        geoms = []
        for feat in src:
            g = geom.shape(feat["geometry"])
            # Reproject each geometry
            g2 = ops.transform(lambda x, y: transformer.transform(x, y), g)
            geoms.append(g2)

        # Merge all geometries and compute area
        merged = ops.unary_union(geoms)
        return merged.area  # in ft²

# Calculate areas
sub_area_ft2 = calc_area(r"E:\Project_deliverable\GIS\Watershed\subbasin_Cypress_Park\Subbasin_CP.shp")
park_area_ft2 = calc_area(r"E:\Project_deliverable\GIS\Park\Cypress_Park_boundaries.shp")

# Convert to acres
sub_area_ac = sub_area_ft2 / 43560
park_area_ac = park_area_ft2 / 43560

print("Subbasin area (ft²):", sub_area_ft2)
print("Subbasin area (acres):", sub_area_ac)
print("Cypress Park area (ft²):", park_area_ft2)
print("Cypress Park area (acres):", park_area_ac)


DriverError: Failed to open dataset (flags=68): E:\Project_deliverable\GIS\Park\Cypress_Park_boundaries.shp